In [15]:
from pyspark.sql import SparkSession

## DEFINE SENSITIVE VARIABLES
POLARIS_URI = 'http://polaris:8181/api/catalog'
POLARIS_CATALOG_NAME = 'polariscatalog'
ENGINEER_CLIENT_ID = 'acb1adbc4ded711c'
ENGINEER_CLIENT_SECRET = 'a4e7cebf9061030b6d18a0c75fb7f811'
HR_CLIENT_ID = 'd2329bb78e4b35e4'
HR_CLIENT_SECRET= 'b7d78348ba458e68ea406a58d6badf1c'
POLARIS_SCOPE = 'PRINCIPAL_ROLE:ALL'


spark = (SparkSession.builder
  .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
  .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.1,org.apache.hadoop:hadoop-aws:3.4.0,software.amazon.awssdk:bundle:2.23.19,software.amazon.awssdk:url-connection-client:2.23.19")
  .config('spark.sql.iceberg.vectorization.enabled', 'false')
         
  # Configure the 'polaris' catalog as an Iceberg rest catalog
  .config("spark.sql.catalog.polaris.type", "rest")
  .config("spark.sql.catalog.polaris", "org.apache.iceberg.spark.SparkCatalog")
  # Specify the rest catalog endpoint       
  .config("spark.sql.catalog.polaris.uri", POLARIS_URI)
  # Enable token refresh
  .config("spark.sql.catalog.polaris.token-refresh-enabled", "true")
  # specify the client_id:client_secret pair
  #.config("spark.sql.catalog.polaris.credential", f"{ENGINEER_CLIENT_ID}:{ENGINEER_CLIENT_SECRET}")
  .config("spark.sql.catalog.polaris.credential", f"{HR_CLIENT_ID}:{HR_CLIENT_SECRET}")
  # Set the warehouse to the name of the catalog we created
  .config("spark.sql.catalog.polaris.warehouse", POLARIS_CATALOG_NAME)

  # Scope set to PRINCIPAL_ROLE:ALL
  .config("spark.sql.catalog.polaris.scope", POLARIS_SCOPE)

  # Enable access credential delegation
  .config("spark.sql.catalog.polaris.header.X-Iceberg-Access-Delegation", 'vended-credentials')

  .config("spark.sql.catalog.polaris.io-impl", "org.apache.iceberg.io.ResolvingFileIO")
  #.config("spark.sql.catalog.polaris.s3.region", "us-west-2")
  .config("spark.history.fs.logDirectory", "/home/iceberg/spark-events")).getOrCreate()

## Start Spark Session
print("Spark Running")


Spark Running


In [16]:
## Tell Spark to use the Polaris catalog
spark.sql("USE polaris")
spark.sql("SHOW NAMESPACES").show()

+-----------+
|  namespace|
+-----------+
|ANDREAS_DEV|
+-----------+



In [5]:
## Create Nested Namespaces
spark.sql("CREATE NAMESPACE IF NOT EXISTS ANDREAS_DEV")
spark.sql("CREATE NAMESPACE IF NOT EXISTS ANDREAS_DEV.PUBLIC")
spark.sql("SHOW NAMESPACES IN ANDREAS_DEV").show()

+------------------+
|         namespace|
+------------------+
|ANDREAS_DEV.PUBLIC|
+------------------+



In [17]:
## Create a table
spark.sql("""CREATE TABLE IF NOT EXISTS ANDREAS_DEV.PUBLIC.engineers (
    id bigint NOT NULL COMMENT 'unique id',
    name string,
    role string)
USING iceberg;
""").show()

++
||
++
++



In [19]:
## Insert Records
spark.sql("INSERT INTO ANDREAS_DEV.PUBLIC.engineers VALUES (1, 'Andreas Kongstad', 'engineer'), (2, 'Carl Bruun','engineer')").show()
spark.sql("SELECT * FROM ANDREAS_DEV.PUBLIC.engineers").show()

++
||
++
++

+---+----------------+--------+
| id|            name|    role|
+---+----------------+--------+
|  1|Andreas Kongstad|engineer|
|  2|      Carl Bruun|engineer|
|  1|Andreas Kongstad|engineer|
|  2|      Carl Bruun|engineer|
|  1|Andreas Kongstad|engineer|
|  2|      Carl Bruun|engineer|
|  1|Andreas Kongstad|engineer|
|  2|      Carl Bruun|engineer|
|  1|Andreas Kongstad|engineer|
|  2|      Carl Bruun|engineer|
+---+----------------+--------+

